# Binoxxo Puzzle

Place X or O in the empty cells so that there are no more than two consecutive X's or O's in a row or a column.
The number of X's is the same as the number of O's in each row and column, and all rows and columns are unique.

Find more Binoxxo puzzles [here](https://www.binoxxo.ch/Binoxxo-Raetselbuch/)

Imports

In [1]:
from ortools.constraint_solver import pywrapcp
from itertools import product
import numpy as np
import itertools

Pretty-print square board

In [2]:
def pretty_print(board):
    mapper = {0: 'O', 1: 'X'}
    for i in range(len(board)):
        for j in range(len(board)):
            print("[{}] ".format(mapper[board[i][j].Value()]), end='')
        print("\n")
    print("\n\n")

Binoxxo puzzle from lecture

In [3]:
binoxxo1 = [
    ["", "1", "", "", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "", ""],
    ["", "1", "1", "", "", "", "", "", "", ""],
    ["", "", "", "", "0", "0", "", "", "", "0"],
    ["1", "", "", "", "", "", "1", "1", "", ""],
    ["", "1", "", "", "1", "", "", "", "", ""],
    ["", "", "", "0", "", "", "1", "", "", ""],
    ["", "0", "", "", "", "", "", "0", "", "0"],
    ["", "", "", "", "0", "", "", "", "", ""],
    ["0", "", "", "", "", "", "", "", "", "0"]
]

And two more examples

In [4]:
binoxxo2 = [
    ["", "", "0", "0", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "", ""],
    ["0", "", "", "", "1", "", "", "1", "", ""],
    ["", "", "", "", "", "0", "", "", "", ""],
    ["0", "", "", "0", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "1", ""],
    ["", "", "", "", "", "", "", "1", "1", ""],
    ["1", "", "", "", "", "", "", "1", "", ""],
    ["", "", "", "", "", "0", "", "", "", "0"],
    ["", "", "", "", "1", "", "0", "", "", ""]
]
binoxxo3 = [
    ["1", "", "", "", "", "", "0", "0", "", ""],
    ["", "", "", "0", "", "", "", "", "1", ""],
    ["", "", "0", "", "", "", "", "", "", ""],
    ["", "", "", "0", "", "1", "", "", "", "0"],
    ["1", "", "1", "", "", "1", "", "", "1", ""],
    ["", "", "1", "", "1", "", "", "", "", ""],
    ["", "", "", "", "1", "1", "", "", "1", ""],
    ["", "", "", "", "", "", "", "", "", "0"],
    ["", "1", "", "1", "", "", "", "", "", ""],
    ["", "1", "", "", "1", "", "", "1", "1", ""]
]

Pick one of the examples

In [5]:
game = binoxxo1

Create constraint solver

In [6]:
solver = pywrapcp.Solver("Binoxxo")

In [7]:
# calculate the board size
n = len(game)
amount_of_x_o = n // 2
# create n*n board for the solution
board = [[solver.IntVar(0, 1) for _j in range(n)] for _i in range(n)]

# assign pre-defined values
for i in range(n):
    for j in range(n):
        if game[i][j] in {"0", "1"}:
            solver.Add(board[i][j] == int(game[i][j] == "1"))
        
# Constraint: The number of X's is the same as the number of O's in each row and column
for i in range(n):
    # if number X has to be the same as O, we always need an even n and the amount of X = O = n / 2
    solver.Add(solver.Sum([board[i][j] for j in range(n)]) == amount_of_x_o)
    solver.Add(solver.Sum([board[j][i] for j in range(n)]) == amount_of_x_o)

# Constraint: and all rows and columns are unique.
all_rows = []
all_cols = []
for i in range(n):
    # use the binary representation inside a row / column to calculate a decimal integer
    # for each row / column which needs to be unique per row / column.
    all_rows.append(solver.Sum([board[i][j] * 2**(j + 1) for j in range(n)]))
    all_cols.append(solver.Sum([board[j][i] * 2**(j + 1) for j in range(n)]))

solver.Add(solver.AllDifferent(all_rows))
solver.Add(solver.AllDifferent(all_cols))

# Constraint: so that there are no more than two consecutive X's or O's in a row or a column.
window = 3
for i in range(n):
    for j in range(n - 2):  # don't overflow the board
        solver.Add(solver.Sum([board[i][j + k] for k in range(window)]) > 0)
        solver.Add(solver.Sum([board[i][j + k] for k in range(window)]) < 3)
        solver.Add(solver.Sum([board[j + k][i] for k in range(window)]) > 0)
        solver.Add(solver.Sum([board[j + k][i] for k in range(window)]) < 3)

Configure solver

In [8]:
# Replace this by a list of all decision variables in your model
all_vars = list(np.concatenate(board))

db = solver.Phase(all_vars, solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [9]:
solver.NewSearch(db)
while solver.NextSolution():
    pretty_print(board)

[X] [X] [O] [X] [O] [O] [X] [O] [O] [X] 

[X] [O] [X] [O] [X] [O] [O] [X] [O] [X] 

[O] [X] [X] [O] [X] [X] [O] [O] [X] [O] 

[X] [X] [O] [X] [O] [O] [X] [O] [X] [O] 

[X] [O] [O] [X] [O] [O] [X] [X] [O] [X] 

[O] [X] [X] [O] [X] [X] [O] [X] [O] [O] 

[O] [O] [X] [O] [O] [X] [X] [O] [X] [X] 

[X] [O] [O] [X] [X] [O] [X] [O] [X] [O] 

[O] [X] [O] [X] [O] [X] [O] [X] [O] [X] 

[O] [O] [X] [O] [X] [X] [O] [X] [X] [O] 






Cleanup

In [10]:
solver.EndSearch()

Print solver information

In [11]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 1
Runtime:   45ms
Failures:  200
Branches:  400 
